In [ ]:
wd<-"/Seurat/reg.CC/WT_replace_v2/res02_220310/"
if(!dir.exists(wd))
    dir.create(wd)
setwd(wd)
getwd()

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
#library(hdf5r)
library(png)
#library(ggalluvial)
#library(tidyverse)
library(clustree)
library(RCTD)
library(Matrix)
#library(SPOTlight)
library(future)
library(grid)
library(RColorBrewer)
#library(pheatmap)
library(circlize)
library(igraph)
#library(ComplexHeatmap)
library(stringr)
#library(ComplexHeatmap)
library(cowplot)
library(biomaRt)
#library(scClustViz)
library(scales)
library(colorspace)
library(biomaRt)
library(clusterProfiler)
options(connectionObserver = NULL)
library(reshape2)
library(corrplot)

In [ ]:
sample_path <- "/data/h5_spatial_20210604/WT/"
f<-dir(sample_path)
f

In [ ]:
f<-f[c(14,13,15,16,6,5,8,7,2,1,3,4,10,9,12,11)]
f

In [ ]:
temp<-Seurat::Load10X_Spatial(paste0(sample_path,f[1]),slice = f[1])
temp

In [ ]:
da<-temp
da
Images(da)

In [ ]:
da$orig.ident<-rep(f[1],dim(da@meta.data)[1])
da$barcode<-paste(da$orig.ident,rownames(da@meta.data),sep = "__")
da$x.axis<-da@images$WT_sham_H_R2_1mm@coordinates$row
da$y.axis<-da@images$WT_sham_H_R2_1mm@coordinates$col
da$imagerow<-da@images$WT_sham_H_R2_1mm@coordinates$imagerow
da$imagecol<-da@images$WT_sham_H_R2_1mm@coordinates$imagecol
da<-RenameCells(da,new.names = da$barcode)
head(da@meta.data)

In [ ]:
for(i in 2:length(f)){
    temp<-Seurat::Load10X_Spatial(paste0(sample_path,f[i]),slice = f[i])
    temp$orig.ident<-rep(f[i],dim(temp@meta.data)[1])
    temp$barcode<-paste(temp$orig.ident,rownames(temp@meta.data),sep = "__")
    temp$x.axis<-temp@images[[f[i]]]@coordinates$row
    temp$y.axis<-temp@images[[f[i]]]@coordinates$col
    temp$imagerow<-temp@images[[f[i]]]@coordinates$imagerow
    temp$imagecol<-temp@images[[f[i]]]@coordinates$imagecol
    temp<-RenameCells(temp,new.names = temp$barcode)
    #head(da@meta.data)
    da<-merge(da,temp)
}
da

In [9]:
da$distance<-NA
da@meta.data[grepl("1mm",da@meta.data$orig.ident,fixed = T),]$distance<-"1mm"
da@meta.data[!grepl("1mm",da@meta.data$orig.ident,fixed = T),]$distance<-"0.5mm"

In [ ]:
Images(da)
saveRDS(da,"WT.merge.replace_v2.raw.rds")

In [4]:
da<-readRDS("WT.merge.replace_v2.raw.rds")

In [ ]:
unique(da$orig.ident)

## split replicates

In [ ]:
colnames(temp@meta.data)

In [ ]:
meta<-temp@meta.data
dim(meta)

- "WT_sham_H_R2_x_not.divide"
- "WT_24h_H_R1_1mm_x_not.divide"
- "WT_24h_T_201231_y_not.divide"

In [ ]:
for (i in unique(da$orig.ident)){  
  tiss <- meta[meta$orig.ident==i,]
  head(tiss)
  #tiss$V3<-as.numeric(tiss$V3)
  #tiss$V4<-as.numeric(tiss$V4)
  x.b<-max(tiss$x.axis)-min(tiss$x.axis)
  h<-hist(tiss$x.axis,breaks =x.b )
  n0<- which(h$counts==0)
  length(n0)
  
  if ( length(n0)==0) {
    print(paste0(i,"_x_not.divide"))
    #not <- paste0(gsub("_tissue_positon.csv","",files[i]),"_x_not.divide")
    #write.table(not,file =paste0(not,".txt") )
  } else {
    x.s<-mean(h$mids[n0])
    s24<-filter(tiss,tiss$x.axis < x.s )
    s13<-filter(tiss,tiss$x.axis > x.s)
    
    y.b13<-max(s13$y.axis)-min(s13$y.axis)
    h<-hist(s13$y.axis,breaks =y.b13)
    n0<- which(h$counts==0)
    
    if (length(n0)==0) {
      print(paste0(i,"_s13_y_not.divide"))
      #not <- paste0(gsub("_tissue_positon.csv","",files[i]),"_y_not.divide")
      #write.table(not,file =paste0(not,".txt") )
    } else {
      
      y.s13<-mean(h$mids[n0]) 
      
      s1<-filter(tiss, tiss$x.axis > x.s & tiss$y.axis > y.s13)
      s1$sample <-"1"
      
      s3<-filter(tiss, tiss$x.axis > x.s & tiss$y.axis < y.s13)
      s3$sample <-"3"      
      
      
    }
      
   y.b24<-max(s24$y.axis)-min(s24$y.axis)
   h<-hist(s24$y.axis,breaks =y.b24)
   n0<- which(h$counts==0)   
   if (length(n0)==0) {
      print(paste0(i,"_s24_y_not.divide"))
      #not <- paste0(gsub("_tissue_positon.csv","",files[i]),"_y_not.divide")
      #write.table(not,file =paste0(not,".txt") )
    } else {
      
      y.s24<-mean(h$mids[n0]) 
      
      s2<-filter(tiss, tiss$x.axis < x.s & tiss$y.axis > y.s24)
      s2$sample <-"2"
      
      s4<-filter(tiss, tiss$x.axis < x.s & tiss$y.axis < y.s24)
      s4$sample <-"4"      
      
      
    }}   
  
    ph.da <- rbind(s1,s2,s3,s4)
    write.csv(ph.da,file = paste0("meta/",i,".replicates_tissue_positon.csv","_meta.csv" ))
    write.csv(s1,file = paste0("meta/",i,"_meta.s1.csv" ))
    write.csv(s2,file = paste0("meta/",i,"_meta.s2.csv" ))
    write.csv(s3,file = paste0("meta/",i,"_meta.s3.csv" ))
    write.csv(s4,file = paste0("meta/",i,"_meta.s4.csv" ))
}

- "WT_sham_H_R2_x_not.divide"
- "WT_24h_H_R1_1mm_x_not.divide"

In [ ]:
for (i in c("WT_sham_H_R2","WT_24h_H_R1_1mm")){  
  tiss <- meta[meta$orig.ident==i,]
  head(tiss)
  #tiss$V3<-as.numeric(tiss$V3)
  #tiss$V4<-as.numeric(tiss$V4)
  y.b<-max(tiss$y.axis)-min(tiss$y.axis)
  h<-hist(tiss$y.axis,breaks =y.b )
  n0<- which(h$counts==0)
  length(n0)
  
  if ( length(n0)==0) {
    print(paste0(i,"_y_not.divide"))
    #not <- paste0(gsub("_tissue_positon.csv","",files[i]),"_x_not.divide")
    #write.table(not,file =paste0(not,".txt") )
  } else {
    y.s<-mean(h$mids[n0])
    s34<-filter(tiss,tiss$y.axis < y.s )
    s12<-filter(tiss,tiss$y.axis > y.s)
    x.b12<-max(s12$x.axis)-min(s12$x.axis)
    h<-hist(s12$x.axis,breaks =x.b12)
    n0<- which(h$counts==0)
    
    if (length(n0)==0) {
      print(paste0(i,"_s12_x_not.divide"))
      #not <- paste0(gsub("_tissue_positon.csv","",files[i]),"_y_not.divide")
      #write.table(not,file =paste0(not,".txt") )
    } else {
      
      x.s12<-mean(h$mids[n0]) 
      
      s1<-filter(tiss, tiss$y.axis > y.s & tiss$x.axis > x.s12)
      s1$sample <-"1"
      
      s2<-filter(tiss, tiss$y.axis > y.s & tiss$x.axis < x.s12)
      s2$sample <-"2" }     
      
      
    
      
   x.b34<-max(s34$x.axis)-min(s34$x.axis)
   h<-hist(s34$x.axis,breaks =x.b34)
   n0<- which(h$counts==0)   
   if (length(n0)==0) {
      print(paste0(i,"_s34_x_not.divide"))
      #not <- paste0(gsub("_tissue_positon.csv","",files[i]),"_y_not.divide")
      #write.table(not,file =paste0(not,".txt") )
    } else {
      
      x.s34<-mean(h$mids[n0]) 
      
      s3<-filter(tiss, tiss$y.axis < y.s & tiss$x.axis > x.s34)
      s3$sample <-"3"
      
      s4<-filter(tiss, tiss$y.axis < y.s & tiss$x.axis < x.s34)
      s4$sample <-"4"      
      
      
    }   }
  
    ph.da <- rbind(s1,s2,s3,s4)
    write.csv(ph.da,file = paste0("meta/",i,".replicates_tissue_positon.csv","_meta.csv" ))
    write.csv(s1,file = paste0("meta/",i,"_meta.s1.csv" ))
    write.csv(s2,file = paste0("meta/",i,"_meta.s2.csv" ))
    write.csv(s3,file = paste0("meta/",i,"_meta.s3.csv" ))
    write.csv(s4,file = paste0("meta/",i,"_meta.s4.csv" ))
}

In [ ]:
f<-dir("/result/Seurat/reg.CC/WT_replace_v2/meta",pattern = "replicate")
f

In [ ]:
rep<-read.csv(paste0("/result/Seurat/reg.CC/WT_replace_v2/meta/",f[1]),header = T)
rownames(rep)<-rep[,1]
rep<-rep[,-1]
head(rep)

for(i in c(2:length(f))){
    tem<-read.csv(paste0("/result/Seurat/reg.CC/WT_replace_v2/meta/",f[i]),header = T)
    rownames(tem)<-tem[,1]
    tem<-tem[,-1]
    tem[1:5,1:5]
    rep<-rbind(rep,tem)
}
dim(rep)
table(rep$sample)

- heatmap of sample bulk correlation

In [ ]:
temp<-readRDS("../WT.merge.replace_v2.raw.rds")
temp

In [ ]:
temp@assays$Spatial@counts[1:5,1:5]

In [ ]:
table(rownames(rep)==colnames(temp))
rep<-rep[colnames(temp),]
table(rownames(rep)==colnames(temp))

In [ ]:
temp@meta.data<-rep
head(temp@meta.data)

In [ ]:
temp$sample<-paste(temp$orig.ident,temp$sample,sep = "_")
table(temp$sample)

In [53]:
temp$sample<-factor(temp$sample,levels = c('WT_sham_H_R2_1mm_1','WT_sham_H_R2_1mm_2','WT_sham_H_R2_1mm_3','WT_sham_H_R2_1mm_4',
         'WT_sham_H_R2_1','WT_sham_H_R2_2','WT_sham_H_R2_3','WT_sham_H_R2_4',
         'WT_sham_T_210323_1','WT_sham_T_210323_2','WT_sham_T_210323_3','WT_sham_T_210323_4',
         'WT_sham_T_R2_1mm_1','WT_sham_T_R2_1mm_2','WT_sham_T_R2_1mm_3','WT_sham_T_R2_1mm_4',
         'WT_3h_H_R2_1mm_1','WT_3h_H_R2_1mm_2','WT_3h_H_R2_1mm_3','WT_3h_H_R2_1mm_4',
         'WT_3h_H_R2_1','WT_3h_H_R2_2','WT_3h_H_R2_3','WT_3h_H_R2_4',
         'WT_3h_T_R2_1','WT_3h_T_R2_2','WT_3h_T_R2_3','WT_3h_T_R2_4',
         'WT_3h_T_210330_1mm_1','WT_3h_T_210330_1mm_2','WT_3h_T_210330_1mm_3','WT_3h_T_210330_1mm_4',
         'WT_24h_H_R1_1mm_1','WT_24h_H_R1_1mm_2','WT_24h_H_R1_1mm_3','WT_24h_H_R1_1mm_4',
         'WT_24h_H_R1_1','WT_24h_H_R1_2','WT_24h_H_R1_3','WT_24h_H_R1_4',
         'WT_24h_T_201231_1','WT_24h_T_201231_2','WT_24h_T_201231_3','WT_24h_T_201231_4',
         'WT_24h_T_R1_1mm_1','WT_24h_T_R1_1mm_2','WT_24h_T_R1_1mm_3','WT_24h_T_R1_1mm_4',
         'WT_72h_H_R1_1mm_1','WT_72h_H_R1_1mm_2','WT_72h_H_R1_1mm_3','WT_72h_H_R1_1mm_4',
         'WT_72h_H_210323_1','WT_72h_H_210323_2','WT_72h_H_210323_3','WT_72h_H_210323_4',
         'WT_72h_T_R2_1','WT_72h_T_R2_2','WT_72h_T_R2_3','WT_72h_T_R2_4',
         'WT_72h_T_R1_1mm_1','WT_72h_T_R1_1mm_2','WT_72h_T_R1_1mm_3','WT_72h_T_R1_1mm_4'))

In [54]:
saveRDS(temp,"WT.merge.replace_v2.raw.rds")

In [11]:
mean_expr<-t(as.matrix(temp@assays$Spatial@counts))
mean_expr<-as.data.frame(mean_expr)

In [ ]:
mean_expr$sample<-temp$sample
mean_expr[1:5,1:5]

In [20]:
mean_expr %>% group_by(sample) %>%
    summarise_all("mean") -> df

In [ ]:
df<-as.data.frame(df)
rownames(df)<-df$sample
df<-df[,-1]
dim(df)
df[1:5,1:5]

In [ ]:
df<-df[c('WT_sham_H_R2_1mm_1','WT_sham_H_R2_1mm_2','WT_sham_H_R2_1mm_3','WT_sham_H_R2_1mm_4',
         'WT_sham_H_R2_1','WT_sham_H_R2_2','WT_sham_H_R2_3','WT_sham_H_R2_4',
         'WT_sham_T_210323_1','WT_sham_T_210323_2','WT_sham_T_210323_3','WT_sham_T_210323_4',
         'WT_sham_T_R2_1mm_1','WT_sham_T_R2_1mm_2','WT_sham_T_R2_1mm_3','WT_sham_T_R2_1mm_4',
         'WT_3h_H_R2_1mm_1','WT_3h_H_R2_1mm_2','WT_3h_H_R2_1mm_3','WT_3h_H_R2_1mm_4',
         'WT_3h_H_R2_1','WT_3h_H_R2_2','WT_3h_H_R2_3','WT_3h_H_R2_4',
         'WT_3h_T_R2_1','WT_3h_T_R2_2','WT_3h_T_R2_3','WT_3h_T_R2_4',
         'WT_3h_T_210330_1mm_1','WT_3h_T_210330_1mm_2','WT_3h_T_210330_1mm_3','WT_3h_T_210330_1mm_4',
         'WT_24h_H_R1_1mm_1','WT_24h_H_R1_1mm_2','WT_24h_H_R1_1mm_3','WT_24h_H_R1_1mm_4',
         'WT_24h_H_R1_1','WT_24h_H_R1_2','WT_24h_H_R1_3','WT_24h_H_R1_4',
         'WT_24h_T_201231_1','WT_24h_T_201231_2','WT_24h_T_201231_3','WT_24h_T_201231_4',
         'WT_24h_T_R1_1mm_1','WT_24h_T_R1_1mm_2','WT_24h_T_R1_1mm_3','WT_24h_T_R1_1mm_4',
         'WT_72h_H_R1_1mm_1','WT_72h_H_R1_1mm_2','WT_72h_H_R1_1mm_3','WT_72h_H_R1_1mm_4',
         'WT_72h_H_210323_1','WT_72h_H_210323_2','WT_72h_H_210323_3','WT_72h_H_210323_4',
         'WT_72h_T_R2_1','WT_72h_T_R2_2','WT_72h_T_R2_3','WT_72h_T_R2_4',
         'WT_72h_T_R1_1mm_1','WT_72h_T_R1_1mm_2','WT_72h_T_R1_1mm_3','WT_72h_T_R1_1mm_4'
         ),]
rownames(df)

In [25]:
write.csv(df,"WT.replace_v2.sample.average.expression.csv")

In [ ]:
write.csv(df,"WT.replace_v2.sample.average.expression.csv")
cor<-cor(t(as.matrix(df)),method = "pearson")
cor<-round(cor,2)
col<-colorRampPalette(c("blue","white","red"))(20)
pdf("WT.replace_v2.sample_bulk.pearson.correlation.pdf")
#par(mar = c(5,5,5,5))
heatmap(x=cor,col=col,scale="none",Colv = NA,Rowv = NA,cexRow = 0.5,cexCol = 0.5,margins = c(6,6)
       )
dev.off()

In [33]:
cor<-cor(t(as.matrix(df)),method = "pearson")

In [34]:
cor<-round(cor,2)

In [ ]:
col<-colorRampPalette(c("blue","white","red"))(20)
pdf("WT.replace_v2.sample_bulk.pearson.correlation.pdf")
#par(mar = c(5,5,5,5))
heatmap(x=cor,col=col,scale="none",Colv = NA,Rowv = NA,cexRow = 0.5,cexCol = 0.5,margins = c(6,6)
       )
dev.off()

#### sample bulk correlation visualization

In [ ]:
source("/script/self_function/cor_diagram.R")
library(ellipse)

In [ ]:
ma<-read.csv("../QC/WT.replace_v2.sample.average.expression.csv")
rownames(ma)<-ma[,1]
ma<-as.matrix(ma[,-1])
dim(ma)

In [ ]:
c<-cor(t(ma),method = "pearson")
c<-round(c,4)
head(c)

In [ ]:
col<-colorRampPalette(rev(brewer.pal(11, "Spectral")))(15)
col
library(scales)
show_col(col)

In [ ]:
#pdf("64samples.average.expression.pearson.correlation.corrplot_220829.pdf",width=40,height=40)
options(repr.plot.width=40,repr.plot.height=40)
corrplot(c,method="shade",type="upper",tl.col = "black",col.lim = c(0,1),
               #lower.col = "#92a8d1",
               col = colorRampPalette(colors = rev(c(#'#5E4FA2',
                                                    '#3E77B5','#48A0B2','#6FC5A4','#A1D9A4')))(100))
#dev.off()

In [ ]:
options(repr.plot.width=12,repr.plot.height=5)
VlnPlot(temp,features = "nCount_Spatial",pt.size = 0,group.by = "sample")+NoLegend()+theme(axis.text.x = element_text(size = 5))
ggsave("QC/nCount.bySample.vlnplot.png",width=12,height=5,dpi = 400)

In [ ]:
unique(temp$orig.ident)

In [59]:
temp$orig.ident<-factor(temp$orig.ident,levels = c('WT_sham_H_R2_1mm','WT_sham_H_R2','WT_sham_T_210323','WT_sham_T_R2_1mm',
                                                  'WT_3h_H_R2_1mm','WT_3h_H_R2','WT_3h_T_R2','WT_3h_T_210330_1mm',
                                                  'WT_24h_H_R1_1mm','WT_24h_H_R1','WT_24h_T_201231','WT_24h_T_R1_1mm',
                                                  'WT_72h_H_R1_1mm','WT_72h_H_210323','WT_72h_T_R2','WT_72h_T_R1_1mm'))

In [ ]:
for(i in c("nCount_Spatial","nFeature_Spatial",'percent.mt',))
options(repr.plot.width=5,repr.plot.height=5)
VlnPlot(temp,features = "nCount_Spatial",pt.size = 0,group.by = "orig.ident")+NoLegend()+theme(axis.text.x = element_text(size = 5))
ggsave("QC/nCount.byIdent.vlnplot.png",width=5,height=5,dpi = 400)

In [ ]:
options(repr.plot.width=12,repr.plot.height=5)
VlnPlot(temp,features = "nFeature_Spatial",pt.size = 0,group.by = "sample")+NoLegend()+theme(axis.text.x = element_text(size = 5))
ggsave("QC/nFeature.bySample.vlnplot.png",width=12,height=5,dpi = 400)

In [ ]:
options(repr.plot.width=5,repr.plot.height=5)
VlnPlot(temp,features = "nFeature_Spatial",pt.size = 0,group.by = "orig.ident")+NoLegend()+theme(axis.text.x = element_text(size = 5))
ggsave("QC/nFeature.byIdent.vlnplot.png",width=5,height=5,dpi = 400)

In [64]:
temp[['percent.mt']]<-PercentageFeatureSet(temp,pattern = "^mt")

In [ ]:
options(repr.plot.width=12,repr.plot.height=5)
VlnPlot(temp,features = "percent.mt",pt.size = 0,group.by = "sample")+NoLegend()+theme(axis.text.x = element_text(size = 5))
ggsave("QC/pt.mt.bySample.vlnplot.png",width=12,height=5,dpi = 400)

In [ ]:
options(repr.plot.width=5,repr.plot.height=5)
VlnPlot(temp,features = "percent.mt",pt.size = 0,group.by = "orig.ident")+NoLegend()+theme(axis.text.x = element_text(size = 5))
ggsave("QC/pt.mt.byIdent.vlnplot.png",width=5,height=5,dpi = 400)

In [ ]:
#remove blood related genes
Hb_gene<-rownames(da)[grepl("^Hb",rownames(da))]
Hb_gene
da<-da[!rownames(da)%in%Hb_gene,colnames(da)]
da

In [6]:
#m.s.genes<-convertHumanGeneList(cc$s.genes)
m.s.genes<-c('Exo1','Msh2','Mcm4','Chaf1b','Rrm2','Cenpu','Mrpl36','Gmnn','Hells','Cdc6','Gins2','Uhrf1',
             'Cdc45','Slbp','Ubr7','Fen1','Rad51ap1','Mcm5','Nasp','Cdca7','Blm','Usp1','Ung','Prim1',
             'Clspn','Mcm6','Dtl','Pola1','Dscc1','Tipin','Wdr76','Casp8ap2','Tyms','Ccne2','Rrm1','Polr1b',
             'Rfc2','Rad51','E2f8','Mcm7','Pcna')
#m.g2m.genes<-convertHumanGeneList(cc$g2m.genes)
m.g2m.genes<-c('Dlgap5','Ctcf','Smc4','Kif20b','Cdc25c','Gtse1','Tpx2','Hmgb2','Cks1brt','Cdca2',
               'Top2a','Cks2','Cdca3','G2e3','Ttk','Ncapd2','Lbr','Anp32e','Ckap2','Tacc3','Nek2',
               'Cenpe','Kif11','Anln','Hjurp','Aurkb','Rangap1','Cks1b','Hmmr','Ckap5','Cdc20',
               'Psrc1','Kif23','Ect2','Kif2c','Ndc80','Nuf2','Cdca8','Birc5','Cenpf','Ube2c','Jpt1',
               'Pimreg','Nusap1','Mki67','Tubb4b','Bub1','Cenpa','Ccnb2','Aurka','Ckap2l')

In [7]:
da<-PercentageFeatureSet(da,"^mt-",col.name = "percent_mt")

In [ ]:
da<-NormalizeData(da)
da<-CellCycleScoring(da,s.features = m.s.genes,g2m.features = m.g2m.genes)
da<-FindVariableFeatures(da,nfeatures = 3000,selection.method ="vst")
da<-ScaleData(da,vars.to.regress = c("S.Score","G2M.Score","nCount_Spatial","percent_mt","orig.ident"),features = rownames(da))

In [9]:
saveRDS(da,"WT.merge.replace_v2.regress_CC.nC.mt.ident.rds")

In [ ]:
#da<-readRDS("WT.merge.replace_v2.spatial_SCT.regress_CC.nC.mt.ident.rds")
da<-readRDS("SCT/WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res025.rds")
da

In [87]:
da$sample<-temp$sample

- quality control and statistic

In [ ]:
slide<-unique(da$orig.ident)
slide

In [ ]:
sum(rowSums(da@assays$Spatial@counts)>0)

In [ ]:
gene_detect<-data.frame(row.names = slide)
##dir.create("QC/")
for(i in slide){
    cells<-rownames(da@meta.data)[da$orig.ident==i]
    temp<-da@assays$Spatial@counts[,cells]
    gene_detect[i,"Total gene detected"]<-sum(rowSums(temp)>0)
    gene_detect[i,"Num of Spots"]<-length(cells)
}
gene_detect
##stats[slide[1],c('nCount_Spatial','nFeature_Spatial','percent_mt','S.Score','G2M.Score')]<-colSums(da@meta.data[da@meta.data$orig.ident==slide[1],c('nCount_Spatial','nFeature_Spatial','percent_mt','S.Score','G2M.Score')])
#stats
#write.csv(stats,"QC/WT.merge.QC_sum.csv")       

In [ ]:
for(i in slide){
    cells<-rownames(da@meta.data)[da$orig.ident==i]
    gene_detect[i,"Num of Spots"]<-length(cells)
}
gene_detect

In [ ]:
stats<-data.frame(row.names = slide)
for(i in slide){
    stats[i,paste0("Mean of ",c('nCount','nFeature','percent_mt','S.Score','G2M.Score'))]<-apply(da@meta.data[da@meta.data$orig.ident==i,c('nCount_Spatial','nFeature_Spatial','percent_mt','S.Score','G2M.Score')],2,mean)
    stats[i,paste0("Median of ",c('nCount','nFeature','percent_mt','S.Score','G2M.Score'))]<-apply(da@meta.data[da@meta.data$orig.ident==i,c('nCount_Spatial','nFeature_Spatial','percent_mt','S.Score','G2M.Score')],2,median)               
}
#stats[slide[1],c('nCount_Spatial','nFeature_Spatial','percent_mt','S.Score','G2M.Score')]<-colSums(da@meta.data[da@meta.data$orig.ident==slide[1],c('nCount_Spatial','nFeature_Spatial','percent_mt','S.Score','G2M.Score')])
stats
#write.csv(stats,"QC/WT.merge.QC_average.csv")   

In [ ]:
stats2<-cbind(stats,gene_detect)
stats2

In [ ]:
write.csv(stats2,"../QC/WT.merge.16sample.QC_220920.csv") 

In [ ]:
stats<-read.csv("../QC/WT.merge.QC_average.csv")
head(stats)
colMeans(stats[2:ncol(stats)])

In [ ]:
options(repr.plot.width=20,repr.plot.height=18)
for(i in c('nCount_Spatial','nFeature_Spatial','percent_mt','S.Score','G2M.Score')){
    p<-SpatialFeaturePlot(da,slot = "counts",features = i,combine=F)
    do.call(ggpubr::ggarrange,c(p,list(ncol=4,nrow=4,common.legend=T,legend="right")))
    ggsave(paste0("QC/WT.merge.regress_CC.nC.mt.ident.spatialQC.",i,".png"),width=20,height=18,dpi = 400)
}


In [ ]:
options(repr.plot.width=12,repr.plot.height=5)
for(i in c('nCount_Spatial','nFeature_Spatial','percent_mt','S.Score','G2M.Score')){
    VlnPlot(da,features = "S.Score",pt.size = 0,group.by = "sample")+NoLegend()+theme(axis.text.x = element_text(size = 5))
    ggsave(paste0("QC/",i,".bySample.vlnplot.png",width=12,height=5,dpi = 400)
}

In [73]:
da$orig.ident<-factor(da$orig.ident,levels = c('WT_sham_H_R2_1mm','WT_sham_H_R2','WT_sham_T_210323','WT_sham_T_R2_1mm',
                                                  'WT_3h_H_R2_1mm','WT_3h_H_R2','WT_3h_T_R2','WT_3h_T_210330_1mm',
                                                  'WT_24h_H_R1_1mm','WT_24h_H_R1','WT_24h_T_201231','WT_24h_T_R1_1mm',
                                                  'WT_72h_H_R1_1mm','WT_72h_H_210323','WT_72h_T_R2','WT_72h_T_R1_1mm'))

In [ ]:
options(repr.plot.width=12,repr.plot.height=5)
for(i in c('nCount_Spatial','nFeature_Spatial','percent_mt','S.Score','G2M.Score')){
    VlnPlot(da,features = "S.Score",pt.size = 0,group.by = "orig.ident")+NoLegend()+theme(axis.text.x = element_text(size = 5))
    ggsave(paste0("QC/",i,".byIdent.vlnplot.png",width=12,height=5,dpi = 400)
}

- add meta info

In [ ]:
da@meta.data$time<-NA
da@meta.data[grepl("sham",da@meta.data$orig.ident,fixed = T),]$time<-"WT_sham"
da@meta.data[grepl("3h",da@meta.data$orig.ident,fixed = T),]$time<-"WT_3h"
da@meta.data[grepl("24h",da@meta.data$orig.ident,fixed = T),]$time<-"WT_24h"
da@meta.data[grepl("72h",da@meta.data$orig.ident,fixed = T),]$time<-"WT_72h"
unique(da$time)

In [89]:
da$time<-factor(da$time,levels = c('WT_sham','WT_3h','WT_24h','WT_72h'))
unique(da$time)

[1] WT_sham WT_3h   WT_24h  WT_72h 
Levels: WT_sham WT_3h WT_24h WT_72h

In [ ]:
da@meta.data$RC<-NA
da@meta.data[grepl("_H_",da@meta.data$orig.ident,fixed = T),]$RC<-"WT_H"
da@meta.data[grepl("_T_",da@meta.data$orig.ident,fixed = T),]$RC<-"WT_T"

unique(da$RC)

In [ ]:
da$pos<-paste(da$RC,da$distance,sep="_")
unique(da$pos)

In [ ]:
da$pos<-factor(da$pos,levels = c('WT_H_1mm','WT_H_0.5mm','WT_T_0.5mm','WT_T_1mm'))
unique(da$pos)

- pca,findneighbor,cluster,UMAP

- determine the 'dimentionality' of the dataset the top pc represent a robust compression of the dataset. how many should choose: resampling test; randomly permute a subset(1%) and return pca, constructing a "null distribution" of features scores, and repeat this procedure. identify 'significant' pcs have strong enrichment of low p-value features.

### SCT strategy

In [3]:
da<-SCTransform(da,assay = "Spatial",
                vars.to.regress = c("S.Score","G2M.Score","nCount_Spatial","percent_mt","orig.ident"),
                return.only.var.genes = F,
               do.scale=T)
da

In [ ]:
mean_expr<-t(as.matrix(da@assays$SCT@counts))
mean_expr<-as.data.frame(mean_expr)

mean_expr$sample<-da$sample
mean_expr[1:5,1:5]

mean_expr %>% group_by(sample) %>%
    summarise_all("mean") -> df

In [14]:
mean_expr %>% group_by(sample) %>%
    summarise_all("mean") -> df

In [ ]:
df<-as.data.frame(df)
rownames(df)<-df$sample
df<-df[,-which(colnames(df)=="sample")]
dim(df)
df[1:5,1:5]

In [ ]:
write.csv(df,"WT.replace_v2.SCT.sample.average.expression.csv")
cor<-cor(t(as.matrix(df)),method = "pearson")
cor<-round(cor,2)
col<-colorRampPalette(c("blue","white","red"))(20)
pdf("WT.replace_v2.SCT.sample_bulk.pearson.correlation.pdf")
#par(mar = c(5,5,5,5))
heatmap(x=cor,col=col,scale="none",Colv = NA,Rowv = NA,cexRow = 0.5,cexCol = 0.5,margins = c(6,6)
       )
dev.off()

In [ ]:
range(da@assays$Spatial@counts)
range(da@assays$SCT@counts)
range(da@assays$Spatial@data)
range(da@assays$SCT@data)
range(da@assays$Spatial@scale.data)
range(da@assays$SCT@scale.data)

In [ ]:
da$orig.ident<-factor(da$orig.ident,levels = c('WT_sham_H_R2_1mm','WT_sham_H_R2','WT_sham_T_210323','WT_sham_T_R2_1mm',
                                              'WT_3h_H_R2_1mm','WT_3h_H_R2','WT_3h_T_R2','WT_3h_T_210330_1mm',
                                              'WT_24h_H_R1_1mm','WT_24h_H_R1','WT_24h_T_201231','WT_24h_T_R1_1mm',
                                              'WT_72h_H_R1_1mm','WT_72h_H_210323','WT_72h_T_R2','WT_72h_T_R1_1mm'))

#### refined vlnplot of QC -220801

In [ ]:
options(repr.plot.width=10,repr.plot.height=6)
VlnPlot(da,cols=colorRampPalette(c('#374E55FF','#DF8F44FF','#00A1D5FF','#B24745FF'))(16),
            features = "nCount_Spatial",
            group.by = "orig.ident",
            pt.size = 0)+
    NoLegend()+
    theme(axis.text.x = element_text(angle = 90))
ggsave("WT.merge.Spatial_nCount.vlnplot_220801.png",width = 10,height = 6,dpi = 300)

In [ ]:
options(repr.plot.width=10,repr.plot.height=6)
VlnPlot(da,cols=colorRampPalette(c('#374E55FF','#DF8F44FF','#00A1D5FF','#B24745FF'))(16),
            features = "nFeature_Spatial",
            group.by = "orig.ident",
            pt.size = 0)+
    NoLegend()+
    theme(axis.text.x = element_text(angle = 90))
ggsave("WT.merge.Spatial_nFeature.vlnplot_220801.png",width = 10,height = 6,dpi = 300)

In [ ]:
options(repr.plot.width=20,repr.plot.height=16)

    ggplot(da@meta.data,aes(x.ad,y.ad))+geom_point(size=1.6,shape=21,stroke=0.15,aes_string(fill="nCount_Spatial"))+
    scale_fill_gradientn(colours = colorRampPalette(rev(brewer.pal(n = 11,name = "Spectral")))(50))+
    xlab(paste0("")) +
    ylab(paste0("")) + 
    theme(panel.background = element_blank(),
          panel.grid.major =element_blank(), 
          panel.grid.minor = element_blank(),
          axis.text.x = element_blank(),#element_text(size = 15,colour = "black"),
          axis.text.y = element_blank()#element_text(size = 20,colour = "black")
          # axis.ticks.x = element_blank(),
          )
ggsave("WT.merge.Spatial_nCount.spatial_220801.png",width = 20,height = 16,dpi = 300)

In [ ]:
options(repr.plot.width=20,repr.plot.height=16)

    ggplot(da@meta.data,aes(x.ad,y.ad))+geom_point(size=1.6,shape=21,stroke=0.15,aes_string(fill="nFeature_Spatial"))+
    scale_fill_gradientn(colours = colorRampPalette(rev(brewer.pal(n = 11,name = "Spectral")))(50))+
    xlab(paste0("")) +
    ylab(paste0("")) + 
    theme(panel.background = element_blank(),
          panel.grid.major =element_blank(), 
          panel.grid.minor = element_blank(),
          axis.text.x = element_blank(),#element_text(size = 15,colour = "black"),
          axis.text.y = element_blank()#element_text(size = 20,colour = "black")
          # axis.ticks.x = element_blank(),
          )
ggsave("WT.merge.Spatial_nFeature.spatial_220801.png",width = 20,height = 16,dpi = 300)

#### injury score

In [33]:
injury_genes<-c('Adamts1','Atf3','Ccl2','Ccnd1','Cd68','Cebpd','Cyba',
                'Fn1','Gal','Gap43','Hmox1','Hspb1','Igfbp2','Jun','Junb',
                'Fos','Lgals1','Neat1','Socs3','Tnc','S100a10','Timp1')

In [36]:
da<-AddModuleScore(da,features = list(injury_genes),name = "injury_score")

In [ ]:
colnames(da@meta.data)

In [ ]:
options(repr.plot.width=10,repr.plot.height=6)
VlnPlot(da,cols=colorRampPalette(c('#374E55FF','#DF8F44FF','#00A1D5FF','#B24745FF'))(16),
            features = "injury_score1",
            group.by = "orig.ident",
            pt.size = 0)+
    NoLegend()+
    theme(axis.text.x = element_text(angle = 90))
ggsave("WT.merge.SCT_22g.injuryScore.vlnplot_220801.png",width = 10,height = 6,dpi = 300)

In [ ]:
options(repr.plot.width=20,repr.plot.height=18)
p<-SpatialFeaturePlot(da,features = "injury_score1",combine=F)
do.call(ggpubr::ggarrange,c(p,list(ncol=4,nrow=4,common.legend=T,legend="right")))
    #ggsave(paste0("QC/WT.merge.regress_CC.nC.mt.ident.spatialQC.",i,".png"),width=20,height=18,dpi = 400)

In [ ]:
options(repr.plot.width=20,repr.plot.height=16)

    ggplot(da@meta.data,aes(x.ad,y.ad))+geom_point(size=1.6,shape=21,stroke=0.15,aes_string(fill="injury_score1"))+
    scale_fill_gradientn(colours = colorRampPalette(rev(brewer.pal(n = 11,name = "Spectral")))(50))+
    xlab(paste0("")) +
    ylab(paste0("")) + 
    theme(panel.background = element_blank(),
          panel.grid.major =element_blank(), 
          panel.grid.minor = element_blank(),
          axis.text.x = element_blank(),#element_text(size = 15,colour = "black"),
          axis.text.y = element_blank()#element_text(size = 20,colour = "black")
          # axis.ticks.x = element_blank(),
          )
ggsave("WT.merge.SCT_22g.injuryScore.spatial_220801.png",width = 20,height = 16,dpi = 300)

- extract SCT.data (gene expressed >0.1% spots) to pyscenic

In [ ]:
SCTdata<-da@assays$SCT@data
head(apply(da@assays$SCT@counts,1,function(x) sum(x>0)))
colnames(da@assays$Spatial@meta.features)
da@assays$SCT@meta.features$expressed_spots<-apply(da@assays$SCT@counts,1,function(x) sum(x>0))
table(da@assays$SCT@meta.features$expressed_spots>20)  
scenic_ma<-da@assays$SCT@data[rownames(da)[da@assays$SCT@meta.features$expressed_spots>20],colnames(da)]
dim(scenic_ma)
write.csv(as.matrix(scenic_ma),"WT.merge.replace_v2.express_cells20.18447g.22820allspots.SCTnormalizedData.csv")

In [ ]:
#da<-RunPCA(da,features = VariableFeatures(da))
da<-FindNeighbors(da,reduction = "pca",dims = 1:20,k.param = 50)
#da<-FindClusters(da,verbose = F)
#da<-RunUMAP(da,reduction = "pca",dims = 1:30)

for(i in c(0.1,0.15,0.2,0.25,0.3)){
    da<-FindClusters(da,resolution = i,verbose = F)
}

da<-RunUMAP(da,reduction = "pca",dims = 1:20)

#options(repr.plot.width=10,repr.plot.height=10)
#clustree(da,prefix="SCT_snn_res.")
#ggsave("WT.merge.SCT.regress_CC.nC.mt.ident.pc30.k20.res_clustree.png",width = 10,height = 10,dpi = 300)
#


In [ ]:
options(repr.plot.width=9,repr.plot.height=7)
DimPlot(da,reduction="umap",label=F,cols = c('#374E55FF','#DF8F44FF','#00A1D5FF','#B24745FF'
),#alpha(c("#1CE6FF","#e6afb9","#A30059","#023fa5"),0.66                                                
        group.by = c("time"))+
theme(#legend.position ="bottom",
    axis.ticks = element_blank(),axis.text = element_blank(),
         axis.line = element_line(size = 0.8,arrow = arrow()) )#+scale_x_discrete(breaks=c())

ggsave("WT.merge.SCT.regress_CC.nC.mt.ident_pc20.k50.time.UMAP.png",width = 9,height = 7,dpi = 400)

In [ ]:
options(repr.plot.width=9,repr.plot.height=7)
DimPlot(da,reduction="umap",label=F,cols = c('#F4A582', '#0571B0'),#alpha(c('#F4A582', '#0571B0'),0.66),
        group.by = c("RC"))+
theme(#legend.position ="bottom",
    axis.ticks = element_blank(),axis.text = element_blank(),
         axis.line = element_line(size = 0.8,arrow = arrow()) )#+scale_x_discrete(breaks=c())

ggsave("WT.merge.SCT.regress_CC.nC.mt.ident_pc20.k50.RC.UMAP.png",width = 9,height = 7,dpi = 400)

In [ ]:
options(repr.plot.width=9,repr.plot.height=7)
DimPlot(da,reduction="umap",label=F,cols = c('#63A79C','#FB9A99'),#alpha(c('#63A79C','#DB95BB'),0.66),
        group.by = c("distance"))+
theme(#legend.position ="bottom",
    axis.ticks = element_blank(),axis.text = element_blank(),
         axis.line = element_line(size = 0.8,arrow = arrow()) )#+scale_x_discrete(breaks=c())

ggsave("WT.merge.SCT.regress_CC.nC.mt.ident_pc20.k50.distance.UMAP.png",width = 9,height = 7,dpi = 400)

In [ ]:
da$domain_res02<-NA
da@meta.data[da@meta.data$SCT_snn_res.0.2 %in% c("0","3","4"),"domain_res02"]<-"WM"
da@meta.data[da@meta.data$SCT_snn_res.0.2 %in% c("1","2"),"domain_res02"]<-"MG"
da@meta.data[da@meta.data$SCT_snn_res.0.2 %in% c("5"),"domain_res02"]<-"DH"
da@meta.data[da@meta.data$SCT_snn_res.0.2 %in% c("6"),"domain_res02"]<-"VH"
unique(da$domain_res02)
table(da$domain_res02)

In [151]:
col_domain<-c('#BC3C29A8','#0072B5A8','#E18727A8','#20854EA8')
names(col_domain)<-c("DH","MG","VH","WM")

In [ ]:
options(repr.plot.width=9,repr.plot.height=7)
DimPlot(da,reduction="umap",label=F,cols = col_domain,#alpha(c("#f79cd4","#1CE6FF","#FFB500","#A4E804"),0.66),
        group.by = c("domain_res02"))+
theme(#legend.position ="bottom",
    axis.ticks = element_blank(),axis.text = element_blank(),
         axis.line = element_line(size = 0.8,arrow = arrow()) )#+scale_x_discrete(breaks=c())

ggsave("WT.merge.SCT.regress_CC.nC.mt.ident_pc20.k50.domain_res02.UMAP.png",width = 9,height = 7,dpi = 400)

In [ ]:
options(repr.plot.width=10,repr.plot.height=10)
clustree(da,prefix="SCT_snn_res.")
ggsave("SCT/WT.merge.SCT.regress_CC.nC.mt.ident.pc20.k50.res_clustree.png",width = 10,height = 10,dpi = 300)

In [ ]:
col<-c('#66AB94','#5D3B83','#316B9B','#E2E293','#C580AA','#C13031','#E18727A8')
show_col(col)
col
names(col)<-c("0","1","2","3","4","5","6")

In [109]:
da$SCT_snn_res.0.2<-factor(da$SCT_snn_res.0.2,levels = c("0","1","2","3","4","5","6"))

In [110]:
Idents(da)<-da$SCT_snn_res.0.2

In [ ]:
options(repr.plot.width=9,repr.plot.height=7)
DimPlot(da,reduction="umap",label=F,cols = col,
        group.by = c("SCT_snn_res.0.2"))+
theme(#legend.position ="bottom",
    axis.ticks = element_blank(),axis.text = element_blank(),
         axis.line = element_line(size = 0.8,arrow = arrow()) )#+scale_x_discrete(breaks=c())

ggsave("WT.merge.regress_CC.nC.mt.ident_SCT.pc20.k50.res02.UMAP.png",width = 9,height = 7,dpi = 400)

In [ ]:
options(repr.plot.width=20,repr.plot.height=18)
p<-SpatialPlot(da,group.by = "SCT_snn_res.0.2",combine = F,cols = col,image.alpha = 0
                  )

do.call(ggpubr::ggarrange,c(p,list(ncol=4,nrow=4,common.legend=T,legend="right")))
ggsave("WT.merge.regress_CC.nC.mt.ident_SCT.pc20.k50.res02.spatial.png",width=20,height=18,dpi=400)

In [251]:
write.csv(da@meta.data,"WT.SCT.pc20.k50.res02.meta.data.csv")

#### manually adjust coordinated and plot spatial cluster

In [ ]:
sample_coor<-da@meta.data[da@meta.data$orig.ident==Images(da)[1],c("x.axis","y.axis")]
#head(sample_coor)
range(sample_coor$x.axis)
range(sample_coor$y.axis)

In [ ]:
meta<-da@meta.data
range(meta$x.axis)
range(meta$y.axis)

In [202]:
meta$x.ad<-NA
meta$y.ad<-NA

In [ ]:
for(i in 1:length(Images(da))){
    m=(i-1)%/%4
    x.add<-(i-4*m-1)*100
    y.add<-150*(4-m)
    meta[meta$orig.ident==Images(da)[i],"x.ad"]<-meta[meta$orig.ident==Images(da)[i],"x.axis"]+x.add
    meta[meta$orig.ident==Images(da)[i],"y.ad"]<-meta[meta$orig.ident==Images(da)[i],"y.axis"]+y.add

}
range(meta$x.ad)
range(meta$y.ad)

In [207]:
da@meta.data<-meta

In [ ]:
meta<-da@meta.data
colnames(meta)

In [ ]:
options(repr.plot.width=20,repr.plot.height=16)
ggplot(meta,aes(x.ad,y.ad,group=SCT_snn_res.0.2))+geom_point(size=1.6,shape=21,stroke=0.15,aes(fill=SCT_snn_res.0.2))+
    scale_fill_manual(values = col)+
    xlab(paste0("")) +
    ylab(paste0("")) + 
    theme(panel.background = element_blank(),
          panel.grid.major =element_blank(), 
          panel.grid.minor = element_blank(),
          axis.text.x = element_blank(),#element_text(size = 15,colour = "black"),
          axis.text.y = element_blank()#element_text(size = 20,colour = "black")
          # axis.ticks.x = element_blank(),
    )
ggsave("WT.merge.regress_CC.nC.mt.ident_SCT.pc20.k50.res02.spatial_manually.png",width=20,height=16,dpi=400)

### manually adjust the coordinates of representative samples -220714

In [73]:
s<-c('WT_sham_H_R2_1mm_2','WT_sham_H_R2_4','WT_sham_T_210323_4','WT_sham_T_R2_1mm_4',
     'WT_3h_H_R2_1mm_2','WT_3h_H_R2_2','WT_3h_T_R2_1','WT_3h_T_210330_1mm_3',
     'WT_24h_H_R1_1mm_3','WT_24h_H_R1_4','WT_24h_T_201231_3','WT_24h_T_R1_1mm_3',
     'WT_72h_H_R1_1mm_2','WT_72h_H_210323_1','WT_72h_T_R2_4','WT_72h_T_R1_1mm_3')

In [ ]:
da_s<-da[rownames(da@assays$SCT@counts),rownames(da@meta.data)[da$sample %in% s]]
da_s

In [76]:
me<-da_s@meta.data

In [87]:
me$x.ad<-0
me$y.ad<-0
for(i in s){
    temp<-me[me$sample==i,c("x.axis","y.axis")]
    xmin<-min(temp$x.axis)
    ymin<-min(temp$y.axis)
    temp$x.ad<-temp$x.axis-xmin
    temp$y.ad<-temp$y.axis-ymin
    me[me$sample==i,"x.ad"]<-temp$x.ad
    me[me$sample==i,"y.ad"]<-temp$y.ad
}

In [ ]:
range(me$x.ad)
range(me$y.ad)

In [ ]:
me$x.ad2<-0
me$y.ad2<-0
for(i in 1:length(s)){
    m=(i-1)%/%4
    x.add<-(i-4*m-1)*40
    y.add<-50*(3-m)
    me[me$sample==s[i],"x.ad2"]<-me[me$sample==s[i],"x.ad"]+x.add
    me[me$sample==s[i],"y.ad2"]<-me[me$sample==s[i],"y.ad"]+y.add

}
range(me$x.ad2)
range(me$y.ad2)

In [ ]:
options(repr.plot.width=10,repr.plot.height=7)
ggplot(me,aes(x.ad2,y.ad2,group=SCT_snn_res.0.2))+geom_point(size=1.6,shape=21,stroke=0.15,aes(fill=SCT_snn_res.0.2))+
    #scale_fill_manual(values = col)+
    xlab(paste0("")) +
    ylab(paste0("")) + 
    theme(panel.background = element_blank(),
          panel.grid.major =element_blank(), 
          panel.grid.minor = element_blank(),
          axis.text.x = element_blank(),#element_text(size = 15,colour = "black"),
          axis.text.y = element_blank()#element_text(size = 20,colour = "black")
          # axis.ticks.x = element_blank(),
    )

In [101]:
write.csv(me,"16samples.meta.ad.220714.csv")

#### domains marker display including well-known and novel in our sham data

In [ ]:
levels(Idents(da))
colnames(da@meta.data)
unique(da$domain)

In [ ]:
Idents(da)<-da$domain_res02
levels(Idents(da))
unique(Idents(da))
table(Idents(da))

In [ ]:
da_sham<-da[rownames(da),rownames(da@meta.data[da@meta.data$time=="WT_sham",])]
da_sham
unique(da_sham$time)

In [ ]:
domain_marker<-FindAllMarkers(da_sham,logfc.threshold = 0.5,min.pct = 0.5,only.pos = T)
head(domain_marker)
dim(domain_marker)

In [ ]:
domain_marker<-domain_marker[domain_marker$p_val_adj<0.01,]
dim(domain_marker)

In [220]:
write.csv(domain_marker,"sham_domain.marker.fc05.pct05.p001.pos.csv")

In [ ]:
domain_marker<-read.csv("sham_domain.marker.fc05.pct05.p001.pos.csv")
head(domain_marker)

In [ ]:
domain_marker %>% 
    group_by(cluster) %>%
    #filter(pct.1-pct.2>0.15) %>%
    top_n(n=20,wt = avg_log2FC) ->top10
top10

In [ ]:
sf<-"domain_marker/"
if(!dir.exists(sf))
    dir.create(sf)
options(repr.plot.width=9,repr.plot.height=7)
for(i in unique(top10$gene)){
    p<-FeaturePlot(da,reduction="umap",features = i,slot = "scale.data",min.cutoff = median(da@assays$SCT@scale.data[i,]))+
        theme(#legend.position ="bottom",
            axis.ticks = element_blank(),axis.text = element_blank(),
            axis.line = element_line(size = 0.8,arrow = arrow()) )#+scale_x_discrete(breaks=c())
    plot(p)
    ggsave(paste0(sf,"WT.merge.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.sham_domain_marker.scale.data.cutMedian.",i,".UMAP_220614.png"),width = 9,height = 7,dpi = 400)
}


#ggsave("SCT/WT.merge.regress_CC.nC.mt.ident_SCT.pc20.k50.res025.UMAP.png",width = 9,height = 7,dpi = 400)

In [160]:
#dir.create("SCT/pc20_k50_res025/domain_marker/")
options(repr.plot.width=20,repr.plot.height=18)
for(i in unique(top10$gene)){
    p<-SpatialFeaturePlot(da,features = i,slot = "scale.data",combine = F,min.cutoff = median(da@assays$SCT@scale.data[i,])
                  )

    do.call(ggpubr::ggarrange,c(p,list(ncol=4,nrow=4,common.legend=T,legend="right")))
    ggsave(paste0(sf,"WT.merge.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.sham_domain_marker.scale.data.cutMedian.",i,".spatial_220614.png"),
           width = 20,height = 18,dpi = 400)
}


In [224]:
saveRDS(da,"WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.rds")

#### cluster marker spatial display -220910

In [3]:
da<-readRDS("WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.rds")

In [ ]:
clusmark<-read.csv("pc20_k50.res02.fc025.pct025.pos.clusterMarker.csv")
rownames(clusmark)<-clusmark[,1]
clusmark<-clusmark[,-1]
head(clusmark)

In [ ]:
clusmark %>% 
    group_by(cluster) %>%
    #filter(pct.1-pct.2>0.15) %>%
    top_n(n=10,wt = avg_log2FC) ->top10
top10

In [ ]:
me<-da@meta.data
temp<-t(as.matrix(da@assays$SCT@data[unique(top10$gene),]))
me<-cbind(me,temp)
colnames(me)

In [ ]:
temp[1:5,1:5]
head(me)

In [7]:
sf<-"cluster_marker/"
if(!dir.exists(sf))
    dir.create(sf)

In [9]:
options(repr.plot.width=20,repr.plot.height=16)
for(i in unique(top10$gene)){
    ggplot(me,aes(x.ad,y.ad))+geom_point(size=1.6,shape=21,stroke=0.15,aes_string(fill=i))+
    scale_fill_gradientn(colours = colorRampPalette(rev(brewer.pal(n = 11,name = "Spectral")))(50))+
    xlab(paste0("")) +
    ylab(paste0("")) + 
    theme(panel.background = element_blank(),
          panel.grid.major =element_blank(), 
          panel.grid.minor = element_blank(),
          axis.text.x = element_blank(),#element_text(size = 15,colour = "black"),
          axis.text.y = element_blank()#element_text(size = 20,colour = "black")
          # axis.ticks.x = element_blank(),
    )
ggsave(paste0(sf,i,".spatial_manually.png"),width=20,height=16,dpi=400)
}